### Toronto Part 3

This is the final part of the 3-part lab on: **Segmenting and Clustering Neighborhoods in Toronto**

In [1]:
# Imports
import pandas as pd
import numpy as np
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans

Prepare our Foursquare variables

In [2]:
# Prepare varibles for Foursquare
CLIENT_ID = '1NUIRXBVBZSLIHU0YZOMDKGLNUN3MIIYO2BJZSKW5032EEJE' # your Foursquare ID
CLIENT_SECRET = 'XIWOVXYYUEAU0Y22UK1ZTPIAHGF0TFOECGJVW4WZ4OQBTGRN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


Define the get_category_type() function

In [3]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

Define the getNearbyVenues() function

In [4]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'Neighbourhood Latitude',
                  'Neighbourhood Longitude',
                  'Venue',
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']

    return(nearby_venues)


Define our columns of interest and read the *toronto_postal_coords.csv* from part 2 of the lab

In [5]:
cols = ['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']
df_toronto = pd.read_csv('toronto_postal_coords.csv', usecols = cols)

print('Shape after reading: {}'.format(df_toronto.shape))
df_toronto.head()

Shape after reading: (103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


Limit our data by filtering on Boroughs that contain 'Toronto'.

In [6]:
# Let's use the subset of Boroughs with 'Toronto' in the name
df_toronto = df_toronto[df_toronto['Borough'].str.contains('Toronto')]

print('Shape after Toronto filter: {}'.format(df_toronto.shape))
df_toronto.head()

Shape after Toronto filter: (38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676845,-79.295225
41,M4K,East Toronto,"The Danforth West,Riverdale",43.683262,-79.355120
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314673
43,M4M,East Toronto,Studio District,43.662766,-79.334830
44,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085


Map our 'Toronto' borough locations.

In [7]:
# Toronto lat lng
toronto_lat = 43.648690000000045
toronto_lng = -79.38543999999996

# Map the the boroughs, acquired from arcgis
map_toronto = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)

map_toronto

Get our Foursquare data using the getNearbyVenues() function.

In [8]:
# Prepare for the Foursquare explore of venues

# Set our limits
LIMIT = 100
radius = 500  # in meters

# Get the full set of Postcode venues
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                 latitudes=df_toronto['Latitude'],
                                 longitudes=df_toronto['Longitude'])



The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

Verify the shape of the dataframe that contains venues.

In [9]:
print('Venues shape: {}'.format(toronto_venues.shape))
toronto_venues.head()

Venues shape: (1739, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676845,-79.295225,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676845,-79.295225,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676845,-79.295225,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.683262,-79.355120,Dollarama,43.686197,-79.355989,Discount Store


How many venues did we retrieve for each neighbourhood?

In [10]:
# How many venues did we get for each Neighbourhood?
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,62,62,62,62,62,62
"Brockton,Exhibition Place,Parkdale Village",66,66,66,66,66,66
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",71,71,71,71,71,71
"Cabbagetown,St. James Town",41,41,41,41,41,41
Central Bay Street,98,98,98,98,98,98
"Chinatown,Grange Park,Kensington Market",95,95,95,95,95,95
Christie,10,10,10,10,10,10


How many unique categories did we receive?

In [11]:
# How many unique categories did we find
print('Unique categories: {}'.format(len(toronto_venues['Venue Category'].unique())))

Unique categories: 212


Prepare the onehot encoding.

In [12]:
# Now we need to prepare the onehot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.head()

,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']

In [14]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [15]:
toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West,Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Prepare the onehot data using the mean(). This will give us 1 row per neighbourhood.

In [16]:
# We need to group the data per neighbourhood so that we have 1 row per neighborhood
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
print('Grouped shape: {}'.format(toronto_grouped.shape))
toronto_grouped.head()

Grouped shape: (38, 213)


,Neighbourhood,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Toy / Game Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.03,0.0,0.010000,0.0,0.000000,0.030000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.010000,0.0,0.000000,0.01,0.0,0.000000
1,Berczy Park,0.0,0.00,0.0,0.016129,0.0,0.000000,0.000000,0.0,0.016129,...,0.0,0.0,0.000000,0.000000,0.016129,0.0,0.000000,0.00,0.0,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.00,0.0,0.015152,0.0,0.015152,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.015152,0.030303,0.0,0.015152,0.00,0.0,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.03,0.0,0.000000,0.0,0.000000,0.020000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.020000,0.0,0.000000,0.00,0.0,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.00,0.0,0.000000,0.0,0.000000,0.014085,0.0,0.000000,...,0.0,0.0,0.014085,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.014085


Show the top 5 most common venues per neighbourhood.

In [17]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Adelaide,King,Richmond----
          venue  freq
0   Coffee Shop  0.09
1          Café  0.06
2         Hotel  0.04
3    Restaurant  0.03
4  Burger Joint  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.08
1    Cocktail Bar  0.05
2      Restaurant  0.05
3          Lounge  0.03
4  Farmers Market  0.03


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0     Coffee Shop  0.11
1            Café  0.06
2          Bakery  0.05
3  Sandwich Place  0.05
4      Restaurant  0.05


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.05
2        Hotel  0.04
3          Bar  0.04
4   Steakhouse  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0         Coffee Shop  0.08
1  Italian Restaurant  0.07
2                Park  0.04
3          Restaurant  0.04
4                Café  0.04


----Cab

In [18]:
# Create a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]


Sort the venues in descending order, using the return_most_common_venues() function. Then view the first 10 neighbourhoods.

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(10)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Asian Restaurant,Restaurant,American Restaurant,Gym,Seafood Restaurant,Gastropub,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Steakhouse,Beer Bar,Cheese Shop,Lounge,Hotel,Seafood Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Furniture / Home Store,Restaurant,Sandwich Place,Bakery,Hotel,Vegetarian / Vegan Restaurant,Beer Bar,Supermarket
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Café,Steakhouse,Hotel,Bar,Restaurant,Japanese Restaurant,Italian Restaurant,Pizza Place,Sushi Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Italian Restaurant,Park,Bar,Café,Restaurant,Gym / Fitness Center,Sandwich Place,Speakeasy,Pub
5,"Cabbagetown,St. James Town",Coffee Shop,Café,Restaurant,Bakery,Pizza Place,Italian Restaurant,Butcher,Breakfast Spot,Jewelry Store,Snack Place
6,Central Bay Street,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Plaza,Tea Room,Sushi Restaurant,Sandwich Place,Fast Food Restaurant,Bookstore,Gym / Fitness Center
7,"Chinatown,Grange Park,Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Chinese Restaurant,Vietnamese Restaurant,Dumpling Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Burger Joint
8,Christie,Café,Grocery Store,Baby Store,Coffee Shop,Italian Restaurant,Athletics & Sports,Playground,Flower Shop,Flea Market,Farm
9,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Dance Studio,Sushi Restaurant,Pub,Burger Joint,Park,Men's Store


Run the K-means clustering using 5 clusters. Then dump the first 10 labels.

In [20]:
# Now we run the K-means clustering

# set number of clusters
kclusters = 5

# Drop the neighbourhood for the clustering process
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3])

Create a dataframe that includes the cluster and the top 10 venues for each neighborhood.

In [21]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head(10) # check the last columns!


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676845,-79.295225,0,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
41,M4K,East Toronto,"The Danforth West,Riverdale",43.683262,-79.355120,0,Bus Line,Park,Discount Store,Grocery Store,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314673,0,Pet Store,Italian Restaurant,Park,Pizza Place,Pub,Movie Theater,Sandwich Place,Fast Food Restaurant,Fish & Chips Shop,Burger Joint
43,M4M,East Toronto,Studio District,43.662766,-79.334830,0,Diner,Café,Bakery,Italian Restaurant,Pizza Place,Sushi Restaurant,Bar,Coffee Shop,Brewery,Arts & Crafts Store
44,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,4,Bus Line,Swim School,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
45,M4P,Central Toronto,Davisville North,43.712815,-79.388526,0,Hotel,Food & Drink Shop,Park,Breakfast Spot,Gym,Clothing Store,Yoga Studio,Farm,Food,Flower Shop
46,M4R,Central Toronto,North Toronto West,43.714523,-79.406960,0,Playground,Park,Gym Pool,Garden,Yoga Studio,Event Space,Food,Flower Shop,Flea Market,Fish Market
47,M4S,Central Toronto,Davisville,43.703395,-79.385964,0,Dessert Shop,Café,Sandwich Place,Italian Restaurant,Pizza Place,Coffee Shop,Thai Restaurant,Sushi Restaurant,Farmers Market,Chinese Restaurant
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.690655,-79.383561,1,Park,Convenience Store,Gym,Tennis Court,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686083,-79.402335,0,Coffee Shop,Light Rail Station,Health & Beauty Service,Supermarket,Liquor Store,Yoga Studio,Food & Drink Shop,Food,Flower Shop,Flea Market


Map the clusters

In [22]:
# Map the clusters

# create map
map_clusters = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters


Review the clusters(0-4) to see how what venues they represent.

In [23]:
# What is cluster 0?
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Neighborhood,Health Food Store,Trail,Pub,Yoga Studio,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
41,East Toronto,0,Bus Line,Park,Discount Store,Grocery Store,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market
42,East Toronto,0,Pet Store,Italian Restaurant,Park,Pizza Place,Pub,Movie Theater,Sandwich Place,Fast Food Restaurant,Fish & Chips Shop,Burger Joint
43,East Toronto,0,Diner,Café,Bakery,Italian Restaurant,Pizza Place,Sushi Restaurant,Bar,Coffee Shop,Brewery,Arts & Crafts Store
45,Central Toronto,0,Hotel,Food & Drink Shop,Park,Breakfast Spot,Gym,Clothing Store,Yoga Studio,Farm,Food,Flower Shop
46,Central Toronto,0,Playground,Park,Gym Pool,Garden,Yoga Studio,Event Space,Food,Flower Shop,Flea Market,Fish Market
47,Central Toronto,0,Dessert Shop,Café,Sandwich Place,Italian Restaurant,Pizza Place,Coffee Shop,Thai Restaurant,Sushi Restaurant,Farmers Market,Chinese Restaurant
49,Central Toronto,0,Coffee Shop,Light Rail Station,Health & Beauty Service,Supermarket,Liquor Store,Yoga Studio,Food & Drink Shop,Food,Flower Shop,Flea Market
50,Downtown Toronto,0,Building,Playground,Park,Bank,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market
51,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Bakery,Pizza Place,Italian Restaurant,Butcher,Breakfast Spot,Jewelry Store,Snack Place


In [24]:
# What is cluster 1?
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Park,Convenience Store,Gym,Tennis Court,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market
64,Central Toronto,1,Park,Construction & Landscaping,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
82,West Toronto,1,Park,Convenience Store,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


In [25]:
# What is cluster 2?
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Home Service,Yoga Studio,Event Space,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


In [26]:
# What is cluster 3?
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,3,Harbor / Marina,Café,Music Venue,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market


In [27]:
# What is cluster 4?
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Bus Line,Swim School,Yoga Studio,Falafel Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
